In [157]:
import time
import pandas as pd
import numpy as np

start = time.time()

with open('data/c_pairs_anon_scored.feb') as f:
    content = f.read().splitlines()
end = time.time()

end - start

60.20700001716614

In [158]:
unique_paths = set()
pairs = []
paths = []
meetings = []

In [159]:
start = time.time()
for i in xrange(len(content)):
    if i % 10000000 == 0:
        print i, ': ', time.time()-start
    (id1, id2, path, count, numMeeting) = content[i].split('|')
    pairs.append(''.join(sorted(list([id1, id2]))))
    paths.append(path)
    meetings.append(int(numMeeting))
unique_paths = set(paths)
end = time.time()
end - start

0 :  0.0
10000000 :  24.8410000801
20000000 :  48.6419999599
30000000 :  74.1759998798
40000000 :  99.4649999142
50000000 :  124.687000036
60000000 :  150.070999861
70000000 :  176.601000071


222.0789999961853

In [160]:
start = time.time()
pdMeetings = pd.Series(meetings, index=pairs, dtype=np.uint8)
end = time.time()
end - start

94.33899998664856

In [5]:
start = time.time()
pdPaths = pd.Series(paths, index=pairs)
end = time.time()
end - start

20.243000030517578

In [6]:
start = time.time()
feats = pd.get_dummies(pdPaths, sparse=True)
end = time.time()
print end - start

471.116000175


In [8]:
start = time.time()
combinedFeats = pd.concat([feats, pdMeetings], axis=1).sample(frac=0.005)
end = time.time()
print end - start

117.612999916


In [9]:
combinedFeats.shape

(393169, 170)

In [ ]:
%xdel combinedFeats

In [10]:
list1 = combinedFeats.columns.tolist()[:169]
list2 = ['numOfMeetings']
list1.extend(list2)
combinedFeats.columns = list1

In [ ]:
# read February t_space data to create label vector

In [11]:
start = time.time()

with open('data/t_pairs_anon_verbs.feb.txt') as f:
    content = f.read().splitlines()
end = time.time()

end - start

47.60800004005432

In [12]:
t_unique_paths = set()
t_pairs = []
t_paths = []
t_interactions = []

In [13]:
start = time.time()
for i in xrange(len(content)):
    (id1, id2, verb, path, count, numInteraction) = content[i].split('|')
    t_pairs.append(''.join(sorted(list([id1, id2]))))
    t_paths.append(path)
    t_interactions.append(int(numInteraction))
t_unique_paths = set(t_paths)
end = time.time()
end - start

0.11500000953674316

In [14]:
ut_pairs = []
ut_paths = []

start = time.time()
for i in xrange(len(t_pairs)):
    if t_pairs[i] not in ut_pairs:
        ut_pairs.append(t_pairs[i])
        ut_paths.append(t_paths[i])
end = time.time()
print end - start

11.6229999065


In [15]:
start = time.time()
pdTPaths = pd.Series(ut_paths, index=ut_pairs)
end = time.time()
end - start

0.01699995994567871

In [16]:
start = time.time()
t_feats = pd.get_dummies(pdTPaths, sparse=True)
end = time.time()
print end - start

0.40499997139


In [17]:
t_labels = pd.Series(1, index=ut_pairs, dtype=np.uint8)

In [18]:
start = time.time()
t_combinedFeats = pd.concat([t_feats, t_labels], axis=1)
end = time.time()
print end - start

list1 = t_combinedFeats.columns.tolist()[:81]
list2 = ['label']
list1.extend(list2)
t_combinedFeats.columns = list1

0.0199999809265


In [19]:
combinedFeats.shape

(393169, 170)

In [20]:
t_combinedFeats.shape

(47931, 82)

In [ ]:
# sample 10%, this hopefully will allow concat next cell
#start = time.time()
#reducedFeats = combinedFeats.sample(frac=0.01)
#end = time.time()
#end - start

In [21]:
# want to know how many positive examples are left?!
start = time.time()
print len(combinedFeats.index.intersection(t_combinedFeats.index))
end = time.time()
end - start

169


0.2090001106262207

In [ ]:
training.shape

In [22]:
start = time.time()
combinedFeats = combinedFeats.to_dense()
t_combinedFeats = t_combinedFeats.to_dense()
training = pd.concat([combinedFeats, t_combinedFeats])
end = time.time()
end - start

0.8539998531341553

In [23]:
training = training.reset_index()

In [24]:
training = training[~training.index.duplicated(keep='first')]

In [25]:
import cPickle as pickle

In [30]:
training = training.fillna(0)

In [207]:
start = time.time()
pickle.dump(training, open('trainingFinal.p', 'wb'))
end = time.time()
end - start

21.10099983215332

In [192]:
training.shape

(440931, 172)

In [77]:
# drop column
# training.drop(['index'], inplace=True,axis=1,errors='ignore')

In [206]:
training.loc[training.label==1,:].shape

(47762, 172)

In [76]:
start = time.time()
training = pickle.load(open('trainingSparse.p', 'rb'))
end = time.time()
end - start

61.704999923706055

In [150]:
training = training.to_dense()

In [173]:
positiveTrainingEx = training.loc[training.label==1,:].index.intersection(pdMeetings.index)

In [189]:
# recover num of meeting for positive training examples
start = time.time()
for hashId in positiveTrainingEx:
    training.at[hashId, 'numOfMeetings'] = float(pdMeetings.at[hashId])

end = time.time()
end - start

0.44300007820129395

In [203]:
training.shape

(440931, 172)

In [204]:
removes = []
import random

start = time.time()
for hashId in training.index:
    if training.at[hashId, 'label'] == 0:
        if random.random() < 0.7:
            removes.append(hashId)
end = time.time()
end - start

training.drop(removes, axis=0, inplace=True)

In [209]:
training.shape

(166113, 172)

In [110]:
# creating test set
start = time.time()
with open('data/c_uniq_pairs_anon_scored.mar') as f:
    content = f.read().splitlines()
end = time.time()
end - start

unique_paths = set()
pairs = []
paths = []
meetings = []

start = time.time()
for i in xrange(len(content)):
    if i % 10000000 == 0:
        print i, ': ', time.time()-start
    (id1, id2, path, count, numMeeting) = content[i].split('|')
    pairs.append(''.join(sorted(list([id1, id2]))))
    paths.append(path)
    meetings.append(int(numMeeting))
unique_paths = set(paths)
end = time.time()
end - start

start = time.time()
pdMeetings = pd.Series(meetings, index=pairs, dtype=np.uint8)
end = time.time()
end - start

start = time.time()
pdPaths = pd.Series(paths, index=pairs)
end = time.time()
end - start

start = time.time()
feats = pd.get_dummies(pdPaths, sparse=True)
end = time.time()
print end - start

start = time.time()
combinedFeatsT = pd.concat([feats, pdMeetings], axis=1)
end = time.time()
print end - start

list1 = combinedFeatsT.columns.tolist()[:181]
list2 = ['numOfMeetings']
list1.extend(list2)
combinedFeatsT.columns = list1

0 :  0.0
10000000 :  20.6529998779
44.8510000706
8.08899998665


In [121]:
overlap = combinedFeatsT.index.intersection(t_feats.index)

In [130]:
ins = []
start = time.time()
for i in xrange(len(combinedFeatsT)):
    if combinedFeatsT.index[i] in overlap:
        ins.append(True)
    else:
        ins.append(False)
end = time.time()
end - start

15.434000015258789

In [137]:
smallFeats = combinedFeatsT.loc[ins, :].to_dense()

In [140]:
pickle.dump(smallFeats, open('smallFeats.p', 'wb'))

In [142]:
pickle.dump(t_feats, open('testPreFeats.p', 'wb'))

In [101]:
start = time.time()
with open('data/t_pairs_anon.mar') as f:
    content = f.read().splitlines()
end = time.time()
end - start

t_unique_paths = set()
t_pairs = []
t_paths = []
t_interactions = []

start = time.time()
for i in xrange(len(content)):
    (id1, id2, path, count, numInteraction) = content[i].split('|')
    t_pairs.append(''.join(sorted(list([id1, id2]))))
    t_paths.append(path)
    t_interactions.append(int(numInteraction))
t_unique_paths = set(t_paths)
end = time.time()
end - start

ut_pairs = []
ut_paths = []

start = time.time()
for i in xrange(len(t_pairs)):
    if t_pairs[i] not in ut_pairs:
        ut_pairs.append(t_pairs[i])
        ut_paths.append(t_paths[i])
end = time.time()
print end - start

start = time.time()
pdTPaths = pd.Series(ut_paths, index=ut_pairs)
end = time.time()
end - start

start = time.time()
t_feats = pd.get_dummies(pdTPaths, sparse=True)
end = time.time()
print end - start

0.113999843597
0.0329999923706
0.0820000171661


In [ ]:
# combine smallFeats and t_feats
# remove unneeded columns
# pickle.dump

In [211]:
smallFeats.index.intersection(t_feats.index)

Index([u'7eb8333dfab5acfb3224ba72', u'1cb285277a19f46500ee1715',
       u'0347cd20bd443d6a92d4982d', u'6e8c29059f1397b1aef473a1',
       u'4c95a5af392062212d0202db', u'bf60d9f80c0bcaabe2d58d01',
       u'3257ca09804066369fb235be', u'd25092dacde2f6a5b409edcc',
       u'159a7006263b95c104571735', u'0c9351b5122b991a39dc33d6',
       ...
       u'0df720d8ea2b84baa7ad92c8', u'699454c2d49ef86564db33cd',
       u'95d07ca6bab3ac784f4ac30d', u'3d6a92d4982d94047ca4d77d',
       u'3f819e80b665eda84483921e', u'2350fe7e568847d5133eec8a',
       u'34e9930c2397699454c2d49e', u'183da0243df0fd7dba25f3c6',
       u'90e9c294ebfed45affb1d292', u'5a06f7669d248b0162c142f5'],
      dtype='object', length=133)

In [212]:
start = time.time()
smallFeats2 = smallFeats.to_dense()
t_feats2 = t_feats.to_dense()
testing = pd.concat([smallFeats2, t_feats2])
end = time.time()
end - start

0.06099987030029297

In [214]:
testing = testing.reset_index()

In [ ]:
testing = testing[~testing['index'].duplicated(keep='first')]

In [252]:
testing = testing.fillna(0)

In [225]:
testing.index = testing['index']

In [227]:
testing.drop(['index'], inplace=True,axis=1,errors='ignore')

In [229]:
testing.columns

Index([u'm', u'mm', u'mmm', u'mmmm', u'mmmmm', u'mmmmmm', u'numOfMeetings',
       u'r', u'rm', u'rmm',
       ...
       u'rrrrrrrrrrrrrrmmmmm', u'rrrrrrrrrrrrrrmmmmmm',
       u'rrrrrrrrrrrrrrmmmmmmm', u'rrrrrrrrrrrrrrmmmmmmmm',
       u'rrrrrrrrrrrrrrmmmmmmmmm', u'rrrrrrrrrrrrrrrmmmmmm',
       u'rrrrrrrrrrrrrrrmmmmmmm', u'rrrrrrrrrrrrrrrmmmmmmmm',
       u'rrrrrrrrrrrrrrrrmmmmm', u'rrrrrrrrrrrrrrrrmmmmmm'],
      dtype='object', length=182)

In [230]:
start = time.time()
pickle.dump(testing, open('testing.p', 'wb'))
end = time.time()
end - start

0.5580000877380371

0.0

In [234]:
len(cols)

164

In [235]:
len(training.columns)

172

In [236]:
a = training[:1]

In [237]:
a

,label,m,mm,mmm,mmmm,mmmmm,mmmmmm,mmmmmmm,numOfMeetings,r,...,rrrrrrrrrrrrmmmmmmmmm,rrrrrrrrrrrrrmmmm,rrrrrrrrrrrrrmmmmm,rrrrrrrrrrrrrmmmmmm,rrrrrrrrrrrrrmmmmmmm,rrrrrrrrrrrrrmmmmmmmm,rrrrrrrrrrrrrmmmmmmmmm,rrrrrrrrrrrrrrmmmmm,rrrrrrrrrrrrrrmmmmmm,rrrrrrrrrrrrrrmmmmmmm
index,,,,,,,,,,,,,,,,,,,,,
6c05124313e2d0bc83d3a8dc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [240]:
testing = pd.concat([testing, a])

In [243]:
testing.drop('6c05124313e2d0bc83d3a8dc', inplace=True)

In [248]:
cols = []
start = time.time()
for col in testing.columns:
    if col not in training.columns:
        cols.append(col)

end = time.time()
end - start

0.0

In [249]:
len(cols)

18

In [250]:
testing.drop(cols, inplace=True,axis=1,errors='ignore')

In [254]:
start = time.time()
pickle.dump(testing, open('testingFinal.p', 'wb'))
end = time.time()
end - start

0.5360000133514404